In [1]:
import os
os.chdir(path='../..')
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import STForte.helper as stfhelper

sc.set_figure_params(dpi=120, transparent=True, dpi_save=400, frameon=False, vector_friendly=False, format="pdf", fontsize=16)
trial_name = "trial-OSCC/sample_1"
palette = px.colors.qualitative.Plotly
plot_dir = f"./{trial_name}/plots"
sc.settings.figdir = plot_dir
plt.rcParams['font.sans-serif'] = [
    'Helvetica',
    'Arial',
    'sans-serif',]

/home/wcx99075523/miniconda3/envs/stforte/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Global seed set to 0


In [2]:
adata = sc.read_h5ad(f"./{trial_name}/outputs/stforte.h5ad")
adata

AnnData object with n_obs × n_vars = 1131 × 15844
    obs: 'pathologist_anno.x', 'sample_id.x', 'cluster_annotations', 'in_tissue', 'row', 'col', 'x', 'y', 'STForte_Mask', 'louvain', 'STForte_anno', 'pathologist_anno'
    uns: 'STForte_anno_colors', 'louvain', 'louvain_colors', 'neighbors', 'pathologist_anno.x_colors', 'pathologist_anno_colors', 'spatial', 'umap'
    obsm: 'STForte_ATTR', 'STForte_COMB', 'STForte_TOPO', 'X_umap', 'spatial'
    layers: 'processed'
    obsp: 'connectivities', 'distances'

In [3]:
adata_sp = sc.read_h5ad(f"./{trial_name}/outputs/sp.h5ad")
adata_sp.uns = adata.uns
adata_sp

AnnData object with n_obs × n_vars = 4097 × 300
    obs: 'spot_instance', 'STForte_anno_with_padding'
    uns: 'STForte_anno_colors', 'louvain', 'louvain_colors', 'neighbors', 'pathologist_anno.x_colors', 'pathologist_anno_colors', 'spatial', 'umap'
    obsm: 'SP_ATTR_FP', 'SP_COMB', 'SP_TOPO', 'X_umap', 'spatial'
    obsp: 'A_STForte_conn', 'A_STForte_recon', 'connectivities', 'distances'

In [4]:
gene_names = ['S100A9','S100A8','S100A7','IGKC']
morans_i, z_score, p_value = stfhelper.Morans_I(adata[:,gene_names].X.A, adata.obsm['spatial'],k=6)
morans_i, z_score, p_value

(array([0.83470796, 0.83599796, 0.80425113, 0.32782528]),
 array([48.48216242, 48.55701817, 46.71507824, 19.07232312]),
 array([0., 0., 0., 0.]))

In [5]:
morans_origin = pd.DataFrame(
    data=dict(samples=['sample_1']*4,genes=gene_names,morans_i=morans_i,z_score=z_score,p_value=p_value)
)
morans_origin.to_csv(f"trial-OSCC/numerical_reasults/morans_origin.csv",mode='a',index=False,header=False)

In [6]:
sp_exp = stfhelper.complete_unseen_expression(
    adata_sp, gene_names, adata, 'SP_TOPO'
)

Propagating genes...:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
pad_names = [i + '_with_padding' for i in gene_names]
morans_i, z_score, p_value = stfhelper.Morans_I(adata_sp.obs[pad_names].to_numpy(), adata_sp.obsm['spatial'],k=6)
morans_i, z_score, p_value

(array([0.94019237, 0.94545366, 0.92835989, 0.74400044]),
 array([103.9977948 , 104.57961099, 102.68931337,  82.30202856]),
 array([0., 0., 0., 0.]))

In [8]:
morans_stforte = pd.DataFrame(
    data=dict(samples=['sample_1']*4,genes=gene_names,morans_i=morans_i,z_score=z_score,p_value=p_value)
)
morans_stforte.to_csv(f"trial-OSCC/numerical_reasults/morans_STforte.csv",mode='a',index=False,header=False)